In [2]:
pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
#pip show openai

Name: openai
Version: 1.52.2
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /Users/adi2/Library/Python/3.12/lib/python/site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: langchain-openai
Note: you may need to restart the kernel to use updated packages.


### Python-dotenv

In [6]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

#os.environ.get('OPENAI_API_KEY')

True

### Chat Models: GPT 3.5 Turbo and GPT 4

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
output = llm.invoke('Explain why sky is blue? ')
print(output.content)

The sky appears blue to us because of the way sunlight is scattered by the Earth's atmosphere. When sunlight reaches the Earth's atmosphere, it is made up of a spectrum of colors, with shorter wavelengths like blue and violet being scattered more easily by the gases and particles in the atmosphere. This scattering causes the blue light to spread out in all directions and fill the sky, giving it a blue appearance. This effect is known as Rayleigh scattering. Since our eyes are more sensitive to blue light, the sky appears blue to us.


In [21]:
from langchain.schema import(
     SystemMessage,
     AIMessage,
     HumanMessage
)

messages = [SystemMessage(content='Respond the answer only in hindi'),
HumanMessage(content='explain why sky is blue in one sentence')]

output = llm.invoke(messages)
print(output.content)

आकाश नीला क्यों है, इसका कारण सूर्य की किरणों का वायुमंडल में छिड़काव होना है।


## Caching LLM Responses

### In-Memory Cache


In [25]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [26]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'tell me a joke that toddlers can understand'
llm.invoke(prompt)


CPU times: user 1.03 s, sys: 562 ms, total: 1.59 s
Wall time: 14.1 s


'\n\nWhy did the teddy bear say no to dessert?\n\nBecause he was already stuffed!'

In [27]:
%%time
llm.invoke(prompt)

CPU times: user 370 μs, sys: 1 μs, total: 371 μs
Wall time: 377 μs


'\n\nWhy did the teddy bear say no to dessert?\n\nBecause he was already stuffed!'

### SQLite Caching

In [28]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))
#first request (not in cache, takes longer)
llm.invoke('tell me a joke')
#second request (cached, faster)
llm.invoke('tell me a joke')

"\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!"

## LLM Streaming

In [33]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write the lyrics of happy birthday, add name aditya to it.'
print(llm.invoke(prompt).content)

Happy birthday to you,
Happy birthday to you,
Happy birthday dear Aditya,
Happy birthday to you.

May your day be filled with joy,
And your heart be light and free,
May all your dreams come true,
And all your wishes be.

Happy birthday dear Aditya,
Happy birthday to you,
Wishing you all the best,
In everything you do.


In [34]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

Happy birthday to you,
Happy birthday to you,
Happy birthday dear Aditya,
Happy birthday to you!

From good friends and true,
From old friends and new,
May good luck go with you,
And happiness too.

## Prompt Templates

In [37]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
template = '''You are an experience virologist. Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)
prompt = prompt_template.format(virus='hiv', language='spanish')
prompt


'You are an experience virologist. Write a few sentences about the following virus "hiv" in spanish.'

In [40]:
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

El VIH, o virus de la inmunodeficiencia humana, es un virus que ataca el sistema inmunológico del cuerpo humano, debilitando la capacidad del organismo para combatir infecciones y enfermedades. Se transmite principalmente a través de relaciones sexuales sin protección, compartir agujas contaminadas o de madre a hijo durante el embarazo, parto o lactancia. Aunque no tiene cura, el VIH puede ser controlado con medicamentos antirretrovirales que permiten a las personas llevar una vida saludable y prolongada.


### ChatPromptTemplates

In [41]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in JSON format. '),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)
messages = chat_template.format_messages(n='10', area='Europe')
print(messages)

[SystemMessage(content='You respond only in JSON format. ', additional_kwargs={}, response_metadata={}), HumanMessage(content='Top 10 countries in Europe by population.', additional_kwargs={}, response_metadata={})]


In [44]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output= llm.invoke(messages)
print(output.content)

{
    "countries": [
        {"country": "Germany", "population": 83149300},
        {"country": "France", "population": 67146000},
        {"country": "United Kingdom", "population": 66040229},
        {"country": "Italy", "population": 60483973},
        {"country": "Spain", "population": 47100396},
        {"country": "Ukraine", "population": 44385155},
        {"country": "Poland", "population": 38433600},
        {"country": "Romania", "population": 19317984},
        {"country": "Netherlands", "population": 17181084},
        {"country": "Belgium", "population": 11589623}
    ]
}


## Simple Chains

In [49]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experience virologist. Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)


chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

output= chain.invoke({'virus': 'HSV', 'language':'Hindi'})




> Entering new LLMChain chain...
Prompt after formatting:
You are an experience virologist. Write a few sentences about the following virus "HSV" in Hindi.

> Finished chain.


In [47]:
print(output)

{'virus': 'HSV', 'language': 'Hindi', 'text': 'HSV (Herpes Simplex Virus) एक बहुत ही सामान्य वायरस है जो आमतौर पर मुंह और योनि क्षेत्र में होता है। यह वायरस लगभग हर व्यक्ति के शरीर में मौजूद होता है, लेकिन कुछ लोगों में इसके लक्षण सामान्य होते हैं जबकि दूसरों में इसके लक्षण गंभीर हो सकते हैं। HSV का कोई इलाज नहीं है, लेकिन दवाओं और उचित देखभाल से इसका प्रबंधन किया जा सकता है।'}


In [51]:
template = 'what is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points'
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

country = input('Enter Country: ')
output = chain.invoke(country)
print(output['text'])

Enter Country:  US




> Entering new LLMChain chain...
Prompt after formatting:
what is the capital of US?. List the top 3 places to visit in that city. Use bullet points

> Finished chain.
The capital of the US is Washington, D.C.

- The White House
- The National Mall (including the Lincoln Memorial, Washington Monument, and Capitol Building)
- Smithsonian Institution (comprising numerous museums and galleries)


## Sequential Chain

In [52]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature = 0.5)
prompt_template1 = PromptTemplate.from_template(
    template = 'you are an experienced scientist and python programmer. write a function that implements the concept of {concept}.'
)
chain1 = LLMChain(llm=llm1, prompt= prompt_template1)

llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature = 1.2)
prompt_template2 = PromptTemplate.from_template(
    template = 'given the python function {function}, describe it as detailed as possible.'
)
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output= overall_chain.invoke('linear regression')



> Entering new SimpleSequentialChain chain...
Sure! Here is a Python function that implements linear regression:

```python
import numpy as np

def linear_regression(X, y):
    # Calculate the number of observations
    n = len(X)
    
    # Calculate the mean of X and y
    mean_x = np.mean(X)
    mean_y = np.mean(y)
    
    # Calculate the slope (m) and y-intercept (b) of the regression line
    numer = 0
    denom = 0
    for i in range(n):
        numer += (X[i] - mean_x) * (y[i] - mean_y)
        denom += (X[i] - mean_x) ** 2
    m = numer / denom
    b = mean_y - (m * mean_x)
    
    return m, b

# Example usage
X = [1, 2, 3, 4, 5]
y = [2, 4, 5, 4, 5]
m, b = linear_regression(X, y)
print(f"The regression line is y = {m}x + {b}")
```

This function takes in two lists, `X` and `y`, representing the independent and dependent variables, respectively. It then calculates the slope and y-intercept of the regression line using the least squares method and returns these values. Finall

In [53]:
print(output['output'])

The provided Python function, `linear_regression`, implements a fundamental operation in statistics and machine learning known as linear regression. This particular implementation calculates the parameters for the best-fitting straight line through a given set of data points defined by lists of x-values (`X`) and corresponding y-values (`y`). This best-fitting line is described by the equation `y = mx + b`, where `m` is the slope of the line and `b` is the y-intercept.

Here's a detailed breakdown of how the function operates:

1. **Import Statement**: The first line, `import numpy as np`, imports the NumPy library, a powerful numerical processing library in Python, renowned for its efficient array operations. The `np.mean()` method from this library is notably utilized within the function.

2. **Function Definition**: The signature `def linear_regression(X, y):` defines the function with two parameters: `X` representing a list of independent variables (inputs), and `y` signifying a li

## LangChain Agents in Action: Python REPL

In [54]:
pip install -q langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [55]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(1, 100) if n%13==0])')

Python REPL can execute arbitrary code. Use with caution.


'[13, 26, 39, 52, 65, 78, 91]\n'

In [56]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)
agent_executor.invoke('Calculate the square root of the factorial of 12 and display it with 4 decimal points')




> Entering new AgentExecutor chain...
To solve this, I need to calculate the factorial of 12 first, then find its square root, and finally format the result to display it with 4 decimal points. I can use the `math` module in Python to accomplish this, as it provides both factorial and square root functions.

Action: Python_REPL
Action Input: import math
Observation: 
Thought:Action: Python_REPL
Action Input: print(f"{math.sqrt(math.factorial(12)):.4f}")
Observation: 21886.1052

Thought:I now know the final answer
Final Answer: 21886.1052

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [57]:
response = agent_executor.invoke('what is answer to 5.1 ** 7.3 ?')



> Entering new AgentExecutor chain...
I need to calculate 5.1 raised to the power of 7.3 to get the answer.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [58]:
response

{'input': 'what is answer to 5.1 ** 7.3 ?', 'output': '146306.05007233328'}

## LangChain Tools: DuckDuckGo and Wikipedia

In [59]:
pip install -q duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [60]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
output = search.invoke('what was the recent spider-man related movie?')
print(output)

Spider-Man 4 is one of the MCU's most anticipated movies and plenty of recent reveals have helped to stoke the excitement. Spider-Man is one of Marvel's most bankable characters, with Spider-Man: No Way Home being the MCU's third-highest-grossing movie overall. With that in mind, it's no surprise that Marvel Studios and Sony are moving ahead with a fourth installment. Disney+ has thwipped up the first footage from Your Friendly Neighborhood Spider-Man.Marvel Studios unveiled a sizzle reel for its upcoming slate of 2025 TV projects, including first looks at the ... Bringing this new version of Peter Parker aka Spider-Man to high-flying life is Hudson Thames (The A-List), who previously voiced the character on Marvel Studios Animation's series, What If…? Link copied to clipboard. There are many upcoming Spider-Man movies and TV shows set to follow Tom Holland's Spider-Man: No Way Home and Spider-Man: Across The Spider-Verse. After 12 Spider-Man movies, the Marvel Universe is still expand

In [61]:
search.name

'duckduckgo_search'

In [62]:
search.description

'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.'

In [64]:
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
output = search.run('Spider-man and Venom')
print(output)

snippet: Warning: This article contains SPOILERS for Venom: The Last Dance. After 6 years of wild, occasionally gross story-telling, Tom Hardy's symbiote trilogy has come to an end with Venom: The Last Dance marking the end of an era.Marvel's most famous symbiote and his host first combined in 2018's Venom, kicking off Sony's Spider-Man Universe with a spirit of uproarious fun that skirted close to an ..., title: Venom: The Last Dance Ending Explained - Sony's Spider-Man Universe ..., link: https://screenrant.com/venom-3-ending-explained/, snippet: Of the three Spider-Man supporting characters featured in these movies, it's kind of surprising that Venom is the one best able to support more than one film. When Venom first appeared in the ..., title: Why 'Venom' Is the Only Sony Spider-Man Spinoff That Works - TIME, link: https://time.com/7098770/venom-the-last-dance-sony-spider-man/, snippet: Till death do they part. Tom Hardy returns in #Venom: The Last Dance - coming exclusively to th

In [65]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region='de-de', max_results=3, safesearch='moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source ='news')
output= search.run('Berlin')

In [66]:
print(output)

snippet: Erfahren Sie mehr über die Geschichte, Geographie, Politik und Kultur der bevölkerungsreichsten und flächengrößten Stadt Deutschlands. Lesen Sie die wichtigsten Fakten, Daten und Sehenswürdigkeiten Berlins auf der Wikipedia-Seite., title: Berlin - Wikipedia, link: https://de.wikipedia.org/wiki/Berlin, snippet: Sie wollen heute ausgehen in Berlin und können sich bei der Vielfalt an Veranstaltungen nicht entscheiden? Die Redaktion gibt hier die besten Veranstaltungstipps für heute aus ausgewählten Kategorien: Events, Kunstausstellungen, Musical- und Konzert-Highlights, Kinderveranstaltungen, Theateraufführungen, Comedy, Kabarett und ..., title: Tagestipps: Heute in Berlin - Berlin.de, link: https://www.berlin.de/tickets/heute/, snippet: Die Dichte der Sehenswürdigkeiten in Berlin-Mitte ist erschlagend. Perfekt für sattelfeste Stadt-Entdecker: Backsteinhäuser und Plattenbauten im Nikolaiviertel, grüne Spielwiese im Großen Tiergarten, Großstadt-Atmosphäre am Potsdamer Platz oder g

In [70]:
import re

# Sample pattern that includes optional whitespace around commas and links
pattern = r'snippet:\s*(.*?),\s*title:\s*(.*?),\s*link:\s*(.*?)\]'

# Apply regex to extract all matches from the output
matches = re.findall(pattern, output, re.DOTALL)

# Print each match in the specified format
for snippet, title, link in matches:
    print(f'Snippet: {snippet}\nTitle: {title}\nLink: {link}\n')
    print('-' * 50)


In [73]:
pip install -q wikipedia

Note: you may need to restart the kernel to use updated packages.


In [75]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [76]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'llamaindex'})

'Page: NebulaGraph\nSummary: NebulaGraph is an open-source distributed graph database built for super '

In [77]:
wiki.invoke('Google Gemini')

'Page: Gemini (chatbot)\nSummary: Gemini, formerly known as Bard, is a generative artificial intellige'

### Creating a ReAct Agent

In [78]:
pip install langchainhub -q

Note: you may need to restart the kernel to use updated packages.


In [79]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [97]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=0)
template = '''
Answer the following questions in Hindi as best as you can.
Questions: {q}
'''

prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull('hwchase17/react')
#print(type(prompt))
#print(prompt.input_variables)
#print(prompt.template)

#1. python REPL tool for executing python code
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name ='Python REPL',
    func= python_repl.run,
    description='Useful when you nedd to use Python to answer a question. You should input Python Code.'
)
    


#2. Wikipedia tool for searching wikipedia
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name ='Wikipedia',
    func =wikipedia.run,
    description ='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

#3. DuckDuckGo Search tool for general web search
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name = 'DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide.'
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    verbose = True,
    handle_parsing_errors = True,
    max_iterations = 10
)










/Users/adi2/Library/Python/3.12/lib/python/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [98]:
#question= 'Generate the first 10 numebers in the fibonacci series.'
#question = 'who is the current president of US'
question= 'Tell me about early life of Mahatma Gandhi'
output= agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
To answer this question in Hindi, I will first find information about Mahatma Gandhi's early life and then translate the key points into Hindi.

Action: Wikipedia
Action Input: Mahatma GandhiPage: Mahatma Gandhi
Summary: Mohandas Karamchand Gandhi (ISO: Mōhanadāsa Karamacaṁda Gāṁdhī; 2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist, and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (Sanskrit: महात्मा, meaning great-souled/venerable), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple in London and was called to the bar in June 1891, at the age of 22. After two uncertain years in India, where he was unable to start a

In [99]:
print(output['input'])


Answer the following questions in Hindi as best as you can.
Questions: Tell me about early life of Mahatma Gandhi



In [100]:
output['output']

'महात्मा गांधी का जन्म 2 अक्टूबर 1869 को गुजरात के एक हिन्दू परिवार में हुआ था। उन्होंने लंदन में इनर टेम्पल में कानून की पढ़ाई की और जून 1891 में, 22 वर्ष की आयु में, बार में बुलाया गया। भारत में दो अनिश्चित वर्षों के बाद, जहाँ वे एक सफल कानूनी अभ्यास शुरू करने में असमर्थ थे, गांधी 1893 में एक भारतीय व्यापारी के मुकदमे का प्रतिनिधित्व करने के लिए दक्षिण अफ्रीका चले गए। वहाँ उन्होंने 21 वर्ष बिताए। दक्षिण अफ्रीका में, गांधी ने एक परिवार की स्थापना की और पहली बार नागरिक अधिकारों के लिए एक अहिंसात्मक प्रतिरोध अभियान में नागरिक अधिकारों के लिए अहिंसात्मक प्रतिरोध का इस्तेमाल किया। 1915 में, 45 वर्ष की आयु में, वे भारत लौटे और जल्द ही किसानों, कृषकों और शहरी मजदूरों को भेदभाव और अत्यधिक भूमि-कर के विरुद्ध प्रदर्शन करने के लिए संगठित करने में जुट गए।'